In [51]:
from science_jubilee.Machine import Machine
from science_jubilee.tools.Tool import Tool
from science_jubilee.decks.Deck import Deck
from science_jubilee.labware.Labware import Labware
from science_jubilee.tools.Pipette import Pipette
import json
import pandas as pd
import numpy as np

## **Tip rack Setting (TODO: Axis configuration)** ##

In [46]:
#m = Machine()
# deck = m.load_deck # We don't have a decj
deck = Deck("lab_automation_deck") # We are not using the deck, so operate any deck.json 
tiprack = deck.load_labware(labware_filename = 'opentrons_96_tiprack_20ul.json', slot = 0)   # Load an opentrons 20uL
tiprack.manual_offset([[269, 221.6],[170, 221.6],[170, 284.6]], save = True)   # Save through list of list. List of coordinates should be A1 A12 H12 subsequently. 
trash = deck.load_labware('agilent_1_reservoir_290ml.json', 1)
samples = deck.load_labware('nest_96_wellplate_200ul_flat.json', slot = 2)
samples.manual_offset([[110,221.6],[11, 221.6], [11, 284.6]], save = True)
stocks = deck.load_labware('corning_6_wellplate_16.8ml.json', slot = 3)
stocks.manual_offset([[103, 25], [23, 25], [23, 64]], save = True)


New manual offset applied to opentrons_96_tiprack_20ul
Manual offset saved
New manual offset applied to nest_96_wellplate_200ul_flat
Manual offset saved
New manual offset applied to corning_6_wellplate_16.8ml_flat
Manual offset saved


In [78]:
def labware_row_wells(labware: Labware):
    labware_row_list, labware_col_list, labware_well_list = labware._create_rows_and_columns()
    labware_to_aspirate_dispense = []
    for row in labware_row_list.keys():
        column_wells = [row + str(col) for col in labware_col_list.keys()]
        labware_to_aspirate_dispense.append(column_wells)
    
    return labware_to_aspirate_dispense

def labware_column_wells(labware: Labware):
    labware_row_list, labware_col_list, labware_well_list = labware._create_rows_and_columns()
    labware_to_aspirate_dispense = []
    for col in labware_col_list.keys():
        row_wells = [row + str(col) for row in labware_row_list.keys()]
        labware_to_aspirate_dispense.append(row_wells)
    
    return labware_to_aspirate_dispense
example_sample = labware_row_wells(samples)[0]
sample_to_dispense = example_sample[2]
print(sample_to_dispense)
print(samples[sample_to_dispense])


A3
Well A3 form NEST 96 Well Plate 200 µL Flat on slot 2


Don't run this (Mocking)

In [6]:
pipette_example = Pipette.from_config(0, 'Pipette', 'P300_config.json')
pipette_example._create_volume_list(volume = [0, 10, 15, 20], total_xfers = 4)

[0.0, 85.31, 127.965, 170.62]

In [131]:
df = pd.DataFrame(np.array([[0, 2, 1, 0], [0, 1, 1, 0], 
                            [0, 0, 1, 0], [1, 0, 1, 0], 
                            [1, 1, 0, 0], [1.5, 1, 0, 0.5], 
                            [0.5, 1, 0, 0]]),
                   columns=['Cyan', 'Magenta', 'Yellow', 'Black'])

# Setting the index to color names
df.index = ['Red', 'Orange', 'Yellow', 'Green', 'Blue', 'Indigo', 'Violet']
# Defining the total volume
total_volume = 100 # Multiply everything by 2 due to Jubilee limitation

# Cyan (A1); Magenta (A2); Yellow(A3); Balck(A4)

# Calculating the new values for each row
for index, row in df.iterrows():
    cyan, magenta, yellow, black =  row['Cyan'], row['Magenta'], row['Yellow'], row['Black']
    
    total_ratio = cyan + magenta + yellow + black
    
    if total_ratio > 0: 
        df.at[index, 'Cyan'] = round((cyan / total_ratio) * total_volume)
        df.at[index, 'Magenta'] = round((magenta / total_ratio) * total_volume)
        df.at[index, 'Yellow'] = round((yellow / total_ratio) * total_volume)
        df.at[index, 'Black'] = round((black / total_ratio) * total_volume)

# Adding a 'Total Volume' column with the specified total volume value
df['Total Volume'] = total_volume
#df['Location'] = well_row

df.reset_index(names = 'color')

,color,Cyan,Magenta,Yellow,Black,Total Volume
0,Red,0.0,67.0,33.0,0.0,100
1,Orange,0.0,50.0,50.0,0.0,100
2,Yellow,0.0,0.0,100.0,0.0,100
3,Green,50.0,0.0,50.0,0.0,100
4,Blue,50.0,50.0,0.0,0.0,100
5,Indigo,50.0,33.0,0.0,17.0,100
6,Violet,33.0,67.0,0.0,0.0,100


In [88]:
def amount_to_aspirate(value):
    new_value = []
    while value > 0:
        if value > 20:
            value -= 20
            new_value.append(20)
        else:
            new_value.append(value)
            value = 0
            
    return new_value

# Testing the code #

In [ ]:
columns_to_extract = ["Cyan", "Magenta", "Yellow", "Black"]

#print(updated_stock_loc)

# Load the pipette to manually set zero_position
#axo.load_tool(P20)

# Initial setting before picking pipette
#P20.add_tiprack(tiprack)
#P20.trash = trash[0]
#axo.pickup_tool(P20)
    
for i, column in enumerate(columns_to_extract):
    # When aspirating individual color(i.e cyan) is done 
    # it goes to next color (i.e Magenta)
    # First we assign all the volumn each column sequentially
    # Pick up the tip in column basis
    #P20.pickup_tip(tiprack[i])
    print(f"Pick the tiprack :{tiprack[i]}")
    # Pipette Tip
    # Stock Location based on column index
    if i < 3:
        stock_location = stocks[labware_row_wells(stocks)[0][i]] 
    else:
        stock_location = stocks[labware_row_wells(stocks)[1][i%3]]

    #column_index = columns_to_extract.index(column)    
    print(f"Now aspirating Stock Color {column}: located at : {stock_location}")
    
    # First column of 96 well plate to be dispensed (Row basis)
    sample_to_dispense = labware_row_wells(samples)[1]
    print(sample_to_dispense)
    for index, row in df.reset_index(names = 'color').iterrows():
        value = row[column]
        # Skip if current volume is 0
        if value == 0:
            print(f"Not using {column} for mixing {row['color']}")
            continue
        # Seperate the volumn
        # Due to the limitation of amount that pipette can aspirate
        seperated_value = amount_to_aspirate(value)
            
        # print(seperated_value)
        for value in seperated_value:
            # Assume we use the same stock colors
            # Add aspiration function from the stock location
            print(f"Now aspirating Stock Color {column}: located at : {stock_location}")
            #P20.aspirate(vol = value, location = stock_location.bottom(9))
            # Add dispensing function to the well plate location
            print(f"Sample dispensed at {samples[sample_to_dispense[index]]}")
            print(f"Sample dispensed volumne: {sample_to_dispense[index]}")
            #P20.dispense(vol = value, location = sample_to_dispense[index].top(-1))
            
        # Drop the tip     
        #P20.drop_tip(trash[0])

# If colormixing is return the pipette to original position
# axo.park_tool()


Pick the tiprack :Well A1 form Opentrons OT-2 96 Tip Rack 20 µL on slot 0
Now aspirating Stock Color Cyan: located at : Well A1 form Corning 6 Well Plate 16.8 mL Flat on slot 3
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12']
Not using Cyan for mixing Red
Not using Cyan for mixing Orange
Not using Cyan for mixing Yellow
Now aspirating Stock Color Cyan: located at : Well A1 form Corning 6 Well Plate 16.8 mL Flat on slot 3
Sample dispensed at Well B4 form NEST 96 Well Plate 200 µL Flat on slot 2
Sample dispensed volumne: B4
Now aspirating Stock Color Cyan: located at : Well A1 form Corning 6 Well Plate 16.8 mL Flat on slot 3
Sample dispensed at Well B4 form NEST 96 Well Plate 200 µL Flat on slot 2
Sample dispensed volumne: B4
Now aspirating Stock Color Cyan: located at : Well A1 form Corning 6 Well Plate 16.8 mL Flat on slot 3
Sample dispensed at Well B5 form NEST 96 Well Plate 200 µL Flat on slot 2
Sample dispensed volumne: B5
Now aspirating Stock Color Cyan: 

## Color Mixing Demo (Column basis)  ##

In [58]:
axo = Machine(address = "192.168.1.2")

In [59]:
axo.home_all()

## **Manual Colormixing** ##

In [134]:
deck = axo.load_deck("lab_automation_deck")
tip_rack = axo.load_labware("opentrons_96_tiprack_20ul", 0)
P20 = Pipette.from_config(0, 'Pipette', 'P300_config.json')
trash = axo.load_labware('agilent_1_reservoir_290ml.json', 1)
#pipette = Pipette(0, "Pipette", brand = "opentron", model = "config", max_volume= 20, min_volume= 0, zero_position = (0,0,0),\
#    blowout_position=None, drop_tip_position = None, mm_to_ul = None)# model = "config")
tiprack.load_manualOffset()
samples = axo.load_labware('nest_96_wellplate_200ul_flat.json', 2)
samples.load_manualOffset()
stocks = axo.load_labware('corning_6_wellplate_16.8ml.json', 3)
stocks.load_manualOffset()
# m.load_tool(pipette) # configure the pipette for use on the m

New manual offset applied to opentrons_96_tiprack_20ul
New manual offset applied to nest_96_wellplate_200ul_flat
New manual offset applied to corning_6_wellplate_16.8ml_flat


In [61]:
axo.move_to(z=200)

In [124]:
axo.load_tool(P20)

In [125]:
axo.pickup_tool(P20)

In [126]:
P20.add_tiprack(tiprack)
P20.trash = trash[0]

In [127]:
P20.pickup_tip(tiprack['A1'])

In [128]:
P20.aspirate(vol = 20, location = stocks['B2'].bottom(15))

In [129]:
P20.dispense(vol = 20, location = samples[11].top(-3), s = 6000)

In [130]:
P20.drop_tip(trash[0])

In [84]:
P20.transfer(200, stocks['A1'].bottom(0), samples[0].top(-1), 
            blowout = True, new_tip='once',
            mix_after = (200, 3))

KeyboardInterrupt: 

In [143]:
P20.pickup_tip(tiprack['A1'])
for _ in range(10):
    P20.aspirate(vol = 20, location = stocks['A1'].bottom(9.5), s = 3000)
    P20.dispense(vol = 20, location = samples[1].top(-1), s= 3000)
P20.drop_tip(trash[0])

In [132]:
axo.park_tool()

## **Color Mixing** ##

In [ ]:
columns_to_extract = ["Cyan", "Magenta", "Yellow", "Black"]

#print(updated_stock_loc)

# Load the pipette to manually set zero_position
axo.load_tool(P20)

# Initial setting before picking pipette
P20.add_tiprack(tiprack)
P20.trash = trash[0]
axo.pickup_tool(P20)
    
for i, column in enumerate(columns_to_extract):
    """When aspirating individual color(i.e cyan) is done 
    it goes to next color (i.e Magenta)
    First we assign all the volumn each column sequentially
    Pick up the tip in column basis"""
    
    P20.pickup_tip(tiprack[i])
    print(f"Pick the tiprack :{tiprack[i]}")
    # Pipette Tip
    
    # Stock Location based on column index
    if i < 3:
        stock_location = stocks[labware_row_wells(stocks)[0][i]] 
    else:
        stock_location = stocks[labware_row_wells(stocks)[1][i%3]]

    #column_index = columns_to_extract.index(column)    
    print(f"Now aspirating Stock Color {column}: located at : {stock_location}")
    
    # First column of 96 well plate to be dispensed (Row basis)
    sample_to_dispense = labware_row_wells(samples)[1]
    
    for index, row in df.reset_index(names = 'color').iterrows():
        value = row[column]
        # Skip if current volume is 0
        if value == 0:
            print(f"Not using {column} for mixing {row['color']}")
            continue
        # Seperate the volumn
        # Due to the limitation of amount that pipette can aspirate
        seperated_value = amount_to_aspirate(value)
            
        # print(seperated_value)
        for value in seperated_value:
            # Assume we use the same stock colors
            # Add aspiration function from the stock location
            print(f"Now aspirating Stock Color {column}: located at : {stock_location}")
            P20.aspirate(vol = value, location = stock_location.bottom(9))
            # Add dispensing function to the well plate location
            print(f"Sample dispensed at {sample_to_dispense[index]}")
            print(f"Sample dispensed volumne: {sample_to_dispense[index]}")
            P20.dispense(vol = value, location = samples[sample_to_dispense[index]].top(-4), s = 3000)
            P20.blowout(s = 3000)
            
        # Drop the tip for the next stock color    
    P20.drop_tip(trash[0])

# If colormixing is return the pipette to original position
axo.park_tool()

Pick the tiprack :Well A1 form Opentrons OT-2 96 Tip Rack 20 µL on slot 0
Now aspirating Stock Color Cyan: located at : Well A1 form Corning 6 Well Plate 16.8 mL Flat on slot 3
Not using Cyan for mixing Red
Not using Cyan for mixing Orange
Not using Cyan for mixing Yellow
Now aspirating Stock Color Cyan: located at : Well A1 form Corning 6 Well Plate 16.8 mL Flat on slot 3
Sample dispensed at B4
Sample dispensed volumne: B4
Now aspirating Stock Color Cyan: located at : Well A1 form Corning 6 Well Plate 16.8 mL Flat on slot 3
Sample dispensed at B4
Sample dispensed volumne: B4
Now aspirating Stock Color Cyan: located at : Well A1 form Corning 6 Well Plate 16.8 mL Flat on slot 3
Sample dispensed at B4
Sample dispensed volumne: B4
Now aspirating Stock Color Cyan: located at : Well A1 form Corning 6 Well Plate 16.8 mL Flat on slot 3
Sample dispensed at B5
Sample dispensed volumne: B5
Now aspirating Stock Color Cyan: located at : Well A1 form Corning 6 Well Plate 16.8 mL Flat on slot 3
Samp

: 